## SVM on Sythetic dataset

In [50]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import time
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [51]:
# Import dataset
df = pd.read_csv('weather_classification_data.csv')
df.shape

(13200, 11)

In [52]:
# Display the first 5 rows of the dataset
df.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [53]:
df.loc[0]

Temperature                      14.0
Humidity                           73
Wind Speed                        9.5
Precipitation (%)                82.0
Cloud Cover             partly cloudy
Atmospheric Pressure          1010.82
UV Index                            2
Season                         Winter
Visibility (km)                   3.5
Location                       inland
Weather Type                    Rainy
Name: 0, dtype: object

In [54]:
X = df.drop(['Weather Type'], axis=1)
y = df['Weather Type']

X_number = X.select_dtypes(include='number') 
X_categorical = X.select_dtypes(exclude='number')
X_categorical = pd.get_dummies(X_categorical, drop_first=False)

In [55]:
X = pd.concat([X_number,X_categorical],axis=1)
X.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km),Cloud Cover_clear,Cloud Cover_cloudy,Cloud Cover_overcast,Cloud Cover_partly cloudy,Season_Autumn,Season_Spring,Season_Summer,Season_Winter,Location_coastal,Location_inland,Location_mountain
0,14.0,73,9.5,82.0,1010.82,2,3.5,False,False,False,True,False,False,False,True,False,True,False
1,39.0,96,8.5,71.0,1011.43,7,10.0,False,False,False,True,False,True,False,False,False,True,False
2,30.0,64,7.0,16.0,1018.72,5,5.5,True,False,False,False,False,True,False,False,False,False,True
3,38.0,83,1.5,82.0,1026.25,7,1.0,True,False,False,False,False,True,False,False,True,False,False
4,27.0,74,17.0,66.0,990.67,1,2.5,False,False,True,False,False,False,False,True,False,False,True


In [56]:
df = pd.concat([X,y], axis=1)

In [57]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Apply SVM

In [ ]:
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])
# param_grid = {
#     'svm__C': [0.001,0.1,1,5,7,10,11,20,50,75,100],
#     'svm__kernel': ['linear', 'rbf'],
#     'svm__gamma': ['scale', 'auto']
# }
# param_grid = {
#     'svm__C': [5,6,7,8,9],
#     'svm__kernel': ['linear', 'rbf'],
#     'svm__gamma': ['scale', 'auto']
# }
param_grid = {
    'svm__C': [5.5,6,6.5],
    'svm__kernel': ['linear', 'rbf'],
    'svm__gamma': ['scale', 'auto']
}
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svm', SVC())]),
             param_grid={'svm__C': [5.5, 6, 6.5],
                         'svm__gamma': ['scale', 'auto'],
                         'svm__kernel': ['linear', 'rbf']})

In [59]:
print("Best parameters:", grid.best_params_)
print("Best cross-validation score:", grid.best_score_)
print("Test set performance:\n")
print(classification_report(y_test, grid.predict(X_test)))

Best parameters: {'svm__C': 6.5, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Best cross-validation score: 0.9088068181818182
Test set performance:

              precision    recall  f1-score   support

      Cloudy       0.88      0.89      0.88       651
       Rainy       0.88      0.89      0.89       647
       Snowy       0.94      0.94      0.94       701
       Sunny       0.93      0.89      0.91       641

    accuracy                           0.91      2640
   macro avg       0.91      0.91      0.91      2640
weighted avg       0.91      0.91      0.91      2640

